Install Required Dependencies

This cell installs all necessary Python packages for the evaluation pipeline
dotenv: For loading environment variables

Langsmith: LangSmith client for dataset management and experiment tracking

requests: HTTP library for API calls

deepeval: Framework for evaluating LLM outputs with various metrics

openai, langchain, langchain-openai: For LLM interactions and chains

langchain_community: Vector database and community integrations


In [ ]:
%pip install dotenv langsmith
%pip install requests
%pip install deepeval==3.6.6
%pip install openai langchain langchain-openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.7/627.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-proto
    Found existing installation: opentelemetry-proto 1.37.0
    Uninstalling opente

Import Core Libraries

Imports essential modules for:
Environment variable management (dotenv)

Azure OpenAI client initialization

LangSmith client for experiment tracking and tracing

Wrapper utilities to integrate OpenAI with LangSmith tracing


In [ ]:
from dotenv import load_dotenv
import os
from openai import AzureOpenAI
from langsmith import Client, traceable
from langsmith.wrappers import wrap_openai

Set LangSmith API Key
Configures the LangSmith API key as an environment variable

This key is required for authenticating with LangSmith services

Used for tracking experiments, storing datasets, and logging evaluation results


In [ ]:
os.environ["LANGSMITH_API_KEY"]  = "*********"   ---->> Add you langsmith API Key

Initialize LangSmith Client
Creates a LangSmith client instance using the API key from environment variables
This client will be used to:
Fetch datasets

Create and manage experiments

Log evaluation results

Confirms successful initialization with a success message


In [ ]:
ls = Client(api_key=os.environ["LANGSMITH_API_KEY"])
print("✅ LangSmith client initialized.")

✅ LangSmith client initialized.


Configure Azure OpenAI Environment
Sets up environment variables for Azure OpenAI service:
- API Key: Authentication credential for Azure OpenAI
- API Base: Endpoint URL for your Azure OpenAI resource
- API Version: Specifies the API version to use
- Deployment Name: The specific GPT model deployment to use

Then initializes the DeepEval Azure OpenAI model wrapper with:
- Temperature=1: Controls randomness in model outputs
- All Azure-specific configuration parameters
This model will be used by DeepEval metrics for evaluation

In [ ]:
import os

# Set environment variables (ensure names match what you will use later)
os.environ["AZURE_OPENAI_API_KEY"]        = "**********"  ---->> Add your Azure OpenAI API Key
os.environ["AZURE_OPENAI_API_BASE"]       = "***********" ----->> Add your Azure OpenAI API Base
os.environ["AZURE_OPENAI_API_VERSION"]    = "***********" ------->> Add your Azure OpenAI API Version
os.environ["AZURE_OPENAI_LLM_DEPLOYMENT"] = "gpt-5-mini"

# Import the model class
from deepeval.models import AzureOpenAIModel

# Initialize the model
azure_client = AzureOpenAIModel(
    model_name="gpt-5-mini",
    deployment_name=os.environ["AZURE_OPENAI_LLM_DEPLOYMENT"],
    azure_openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_API_BASE"],
    temperature=1
)


Fetch and Display Dataset from LangSmith
Purpose: Verify dataset contents before running evaluation
Steps:
1. Connect to LangSmith using API key
2. Fetch all examples from the specified dataset ID
3. Loop through each example and display:
    - Question: The input query   
    -Actual Answer: The model's response
   - Expected Answer: The ground truth/reference answer

This is useful for:
- Verifying dataset structure
- Checking data quality before evaluation
- Understanding the format of inputs and outputs

In [ ]:
# ✅ Imports
import os
import pandas as pd
from langsmith import Client
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric, SummarizationMetric, ToxicityMetric
from tabulate import tabulate
from IPython.display import display, HTML

# -----------------------------
# 1️⃣ Connect to LangSmith
# -----------------------------
client = Client(api_key="****************") ----->> Add here langsmithe API key
dataset_id = "*****************" -------->> Add here dataset ID

# Fetch all examples
examples = client.list_examples(dataset_id=dataset_id)

# Loop through all examples and print their content
for ex in examples:
    question = ex.inputs.get("Question", "")
    actual_answer = ex.outputs.get("Actual_Answer", "")
    expected_answer = ex.outputs.get("Expected_Answer", "")

    print("Question:", question)
    print("Actual Answer:", actual_answer)
    print("Expected Answer:", expected_answer)
    print("-" * 50)



Question: How many Sick Leave (SL) days are provided annually and when is a medical certificate required?
Actual Answer: Employees have 12 casual leaves per year; no certificate required.
Expected Answer: 7 days per year; a medical certificate is required for absences exceeding three consecutive days; unused sick leave lapses at the end of the year.
--------------------------------------------------
Question: What is the paternity leave entitlement and within what timeframe must it be taken?
Actual Answer: Male employees get 5 days paid paternity leave, to be taken within 15 days of childbirth.
Expected Answer: Male employees are eligible for 5 days of paid paternity leave, which must be availed within 15 days of childbirth.
--------------------------------------------------
Question: How does the policy address dress on festival days and Casual Fridays?
Actual Answer: Employees can wear ethnic clothes on festivals; casual Fridays allow jeans and t-shirts.
Expected Answer: On festival 

Run DeepEval Metrics and Save Results
This is the MAIN EVALUATION CELL that performs the complete evaluation pipeline
Process:
CONNECT TO LANGSMITH
- Initializes client and fetches dataset examples
- Prepares evaluation cases with input, actual output, and expected output
SELECT METRICS
- AnswerRelevancyMetric: Measures how relevant the answer is to the question
 -SummarizationMetric: Evaluates quality of summarization
 - ToxicityMetric: Detects toxic or harmful content

DEFINE EVALUATION FUNCTION
Creates LLMTestCase for each example
Runs all metrics on each test case
Captures scores and reasoning for each metric
 Displays results in HTML table format

EVALUATE ALL EXAMPLES
- Loops through each example in the dataset
- Runs evaluate_case() for each
- Collects all results in all_results_list

SAVE RESULTS TO CSV
 - Converts results to DataFrame
- Saves to CSV file for later use
- Each row contains: Question, Metric, Score, Reason, Actual_Answer, Expected_Answer


In [ ]:
# ✅ Imports
import os
import pandas as pd
from langsmith import Client
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric, SummarizationMetric, ToxicityMetric
from tabulate import tabulate
from IPython.display import display, HTML

# -----------------------------
# 1️⃣ Connect to LangSmith
# -----------------------------
client = Client(api_key="****************") ----->> Add here langsmithe API key
dataset_id = "*****************" -------->> Add here dataset ID


# Fetch all examples
examples = client.list_examples(dataset_id=dataset_id)


# Prepare evaluation cases
eval_cases = [
    {
        "input": ex.inputs.get("Question", ""),
        "actual_output": ex.outputs.get("Actual_Answer", ""),
        "expected_output": ex.outputs.get("Expected_Answer", "")
    }
    for ex in examples
]


# -----------------------------
# 2️⃣ Pick metrics
# -----------------------------
metrics_list = [AnswerRelevancyMetric, SummarizationMetric, ToxicityMetric]

# -----------------------------
# 3️⃣ Evaluate function
# -----------------------------
def evaluate_case(case_data, model=None, metrics_list=None):
    if metrics_list is None or model is None:
        return

    test_case = LLMTestCase(
        input=case_data["input"],
        expected_output=case_data["expected_output"],
        actual_output=case_data["actual_output"],
        retrieval_context=None,
        context=None
    )

    all_results = []
    for metric in metrics_list:
        try:
            m = metric(model=model)
            m.measure(test_case)
            all_results.append({
                "Metric": type(m).__name__,
                "Score": m.score,
                "Reason": getattr(m, "reason", None)
            })
        except Exception as e:
            all_results.append({
                "Metric": type(metric).__name__,
                "Score": None,
                "Reason": f"Error: {e}"
            })

    print(f"\n📘 Results for: {case_data['input']}\n")
    display(HTML(tabulate(
        [[r["Metric"], r["Score"], r["Reason"]] for r in all_results],
        headers=["Metric", "Score", "Reason"],
        tablefmt="html"
    )))

    return all_results


# -----------------------------
# 4️⃣ Evaluate all examples
# -----------------------------
all_results_list = []

for case in eval_cases:
    results = evaluate_case(case, model=azure_client, metrics_list=metrics_list)
    for r in results:
        all_results_list.append({
            "Question": case["input"],
            "Expected_Answer": case["expected_output"],
            "Actual_Answer": case["actual_output"],
            "Metric": r["Metric"],
            "Score": r["Score"],
            "Reason": r["Reason"]
        })

# -----------------------------
# 5️⃣ Save results to CSV (optional)
# -----------------------------
df_results = pd.DataFrame(all_results_list)
df_results.to_csv("deepeval_results_updated_final.csv", index=False)
print("✅ Results saved to deepeval_results.csv")


Output()

Output()

Output()


📘 Results for: How many Sick Leave (SL) days are provided annually and when is a medical certificate required?



Metric,Score,Reason
AnswerRelevancyMetric,0,"The score is 0.00 because the response discussed casual leave entitlement and certificate rules for casual leave rather than answering how many Sick Leave (SL) days are provided annually or when a medical certificate is required for SL. Since none of the requested SL details were provided, the answer could not be scored higher."
SummarizationMetric,0,"The score is 0.00 because the summary introduces unsupported claims (mentions casual leave and a ’12 days’ figure and certificate rules for casual leave) that do not appear in the original, and it omits answers to explicit points the original raises (whether it asks how many Sick Leave days are provided annually, when a medical certificate is required, whether it mentions Sick Leave, whether it is phrased as a question, and whether it asks about both SL days and certificate requirements). The summary therefore adds extra information and fails to address the original’s questions, making it misleading and unusable."
ToxicityMetric,0,"The score is 0.00 because the output contains no abusive or derogatory language and uses polite, constructive phrasing (for example, ""thank you"" and ""please let me know""), demonstrating a respectful, helpful tone."


...

Output()

Output()


📘 Results for: What footwear is considered acceptable and what specific restrictions are mentioned?



Metric,Score,Reason
AnswerRelevancyMetric,1,"The score is 1.00 because the response fully addressed which footwear is acceptable and the specific restrictions with no irrelevant statements; there was nothing extraneous to trim. It can’t be higher because 1.00 is the maximum, and the answer earned that top score."
SummarizationMetric,0,"The score is 0.00 because the summary asserts specific permitted (formal shoes, sandals, juttis) and forbidden (very high heels, flip-flops) footwear that the original never states, directly contradicting the source which only asks which footwear is acceptable and what restrictions exist. The summary also fails to report simple verifiable points the original does answer (that the text is phrased as a question and that it includes the word ""footwear""), so it both fabricates details and omits basic facts, making it unreliable."
ToxicityMetric,0,"The score is 0.00 because the response contains no abusive or hateful language and instead uses polite, constructive wording (for example, ""clear and respectful explanation""), indicating a helpful, non‑toxic tone."


✅ Results saved to deepeval_results.csv


Upload Evaluation Results to LangSmith UI
This cell uploads the saved DeepEval results to LangSmith for visualization
Purpose: Display all evaluation metrics in the LangSmith web interface
deepeval_wrapper(): Takes inputs (question) and returns outputs (answer + metrics)
Looks up results from CSV for each question
Returns formatted output with all metric scores and reasons

DEFINE EVALUATOR FORMATTERS
- answer_relevancy_evaluator: Extracts AnswerRelevancyMetric score
- summarization_evaluator: Extracts SummarizationMetric score
- toxicity_evaluator: Extracts ToxicityMetric score
- Each evaluator returns: key (metric name), score (float), comment (reason)

 RUN EVALUATION FORMATTER & UPLOAD
- Uses ls_client.evaluate() to create experiment in LangSmith
- Processes each example with the wrapper function
- Applies all three evaluators to capture scores
- Uploads results to LangSmith dashboard
Result: All metrics visible in LangSmith UI with scores and explanations


In [ ]:
# ✅ Imports
import os
import pandas as pd
from langsmith import Client
from datetime import datetime

# -----------------------------
# 1️⃣ Connect to LangSmith
# -----------------------------
client = Client(api_key="****************") ----->> Add here langsmithe API key
dataset_id = "*****************" -------->> Add here dataset ID
dataset = ls_client.read_dataset(dataset_id=dataset_id)
print(f"✅ Using existing dataset: {dataset.name} (ID: {dataset.id})")

# -----------------------------
# 3️⃣ Load DeepEval results from CSV
# -----------------------------
df_results = pd.read_csv("/content/deepeval_results_updated_final.csv")
print(f"✅ Loaded {len(df_results)} results from CSV")
print(f"📊 Unique questions: {df_results['Question'].nunique()}")

# -----------------------------
# 4️⃣ Prepare model wrapper function
# -----------------------------
def Resultslogback_wrapper(inputs: dict) -> dict:
    """
    Returns actual answer and all metrics for a given question.
    """
    question = inputs.get("Question", "")
    rows = df_results[df_results["Question"] == question]

    if rows.empty:
        return {"answer": None, "metrics": {}}

    actual_answer = rows.iloc[0]["Actual_Answer"]
    metrics = {}
    for _, row in rows.iterrows():
        metrics[row["Metric"]] = {"Score": row["Score"], "Reason": row["Reason"]}

    return {"answer": actual_answer, "metrics": metrics}

# -----------------------------
# 5️⃣ Dynamic evaluator for all metrics
# -----------------------------
def dynamic_metrics_evaluator(run, example):
    outputs = run.outputs
    metrics = outputs.get("metrics", {})

    results = []
    for metric_name, data in metrics.items():
        results.append({
            "key": metric_name.lower(),
            "score": float(data.get("Score", 0.0)),
            "comment": data.get("Reason", "")
        })
    return results


# -----------------------------
# 6️⃣ Run evaluation and upload to LangSmith
# -----------------------------
experiment_name = f"deepeval-metrics-{datetime.now().strftime('%Y%m%d_%H%M%S')}"
print(f"\n📤 Starting LangSmith evaluation: {experiment_name}\n")

results = ls_client.evaluate(
    Resultslogback_wrapper,      # Model wrapper that returns answer + all metrics
    data=dataset_id,             # Dataset ID
    evaluators=[dynamic_metrics_evaluator],
    experiment_prefix=experiment_name,
    description="DeepEval metrics uploaded dynamically",
    max_concurrency=1,
    num_repetitions=1,
)

print("\n✅ Evaluation complete!")
print(f"📊 Results object: {results}")


✅ Using existing dataset: Policy Q&A (ID: 444837f8-409d-481a-a39b-80e11c0d5cef)


FileNotFoundError: [Errno 2] No such file or directory: '/content/deepeval_results_updated_final.csv'